In [ ]:
import pandas as pd
import numpy as np

# Блокнот для преобразования базовых запросов в пользовательские сценарии.

Необходим, если статистика была собрана по endpoint`ам, а пользовательские сценарии создавались отдельно. 
Рассчитывает количество пользовательских сценариев, исходя из статистики по базовым запросам.

В `uc_data.csv` дожна находиться таблица следующего формата:

- UC — Пользовательский сценарий.
- КЗ — Сколько раз вызвана запрос при выполнении пользовательского сценария(UC)
- Запрос — endpoint, например: `GET_/app/info`.

|name      |UC01                    |UC02|UC*n*|
|:---------|:-----------------------|:---|:----|
|Запрос 1  |Количество запросов (КЗ)|КЗ  |КЗ   |
|Запрос 2  |КЗ                      |КЗ  |КЗ   |
|Запрос *n*|КЗ                      |КЗ  |КЗ   |

Файл `load_profile.csv` берется из предыдущего блокнота.

## Читаем файлы данных:

In [ ]:
uc_data = pd.read_csv("uc_data.csv", sep=",")
count_data = pd.read_csv("load_profile.csv", sep=",")

## Объединяем в один DataFrame:

In [ ]:
result_data = uc_data.set_index("name").join(count_data[["name", "count"]].set_index("name"))
result_data

## Удаляем дубликаты:

In [ ]:
data_without_duplicates = result_data.drop_duplicates(subset=result_data.columns.to_list()[:-1], keep="first")
data_without_duplicates

## Сохраняем DataFrame для вычисления обратной матрицы:

In [ ]:
sub_data = data_without_duplicates.iloc[:, :-1]
sub_data

## Вычисляем обратную матрицу:

In [ ]:
sub_data_inv = pd.DataFrame(np.linalg.pinv(sub_data.values), sub_data.columns, sub_data.index)
sub_data_inv

## Вычисляем количество пользовательских сценариев:
Может возникнуть ситуация, когда количество пользовательских сценариев будет отрицательным. Это означает, что запросы не удалось разбить по пользовательским сценарям.
Чтобы решить данную проблему, можно изменить количество пользовательских сценариев в большую или меньшую сторону.

In [ ]:
result = sub_data_inv.dot(data_without_duplicates.iloc[:, -1:])
result

## Отбрасываем цифры после запятой, если это необходимо:

In [ ]:
result["count"] = result["count"].astype("int32")
result